# 📌 MongoDB: Agregaciones y Pipeline

### 1. Introducción a las agregaciones

MongoDB permite **procesar y transformar documentos** mediante pipelines de agregación, que son **secuencias de etapas** (`stages`) aplicadas sobre los datos.  
Cada etapa recibe los resultados de la anterior y puede: filtrar, agrupar, ordenar, proyectar, sumar, etc.

---

### 2. Operadores de agregación

| Operador | Significado | Mongo Shell | Python (`pymongo`) |
|----------|------------|------------|------------------|
| `$match` | Filtrar documentos | `{ $match: { estatus: "Pendiente" } }` | `{"$match": {"estatus": "Pendiente"}}` |
| `$group` | Agrupar documentos | `{ $group: { _id: "$estatus", total: { $sum: 1 } } }` | `{"$group": {"_id": "$estatus", "total": {"$sum": 1}}}` |
| `$sum` | Suma de valores | `{ $sum: "$cantidad" }` | `{"$sum": "$cantidad"}` |
| `$avg` | Promedio de valores | `{ $avg: "$precio" }` | `{"$avg": "$precio"}` |
| `$max` | Valor máximo | `{ $max: "$precio" }` | `{"$max": "$precio"}` |
| `$min` | Valor mínimo | `{ $min: "$precio" }` | `{"$min": "$precio"}` |
| `$first` | Primer valor del grupo | `{ $first: "$cliente" }` | `{"$first": "$cliente"}` |
| `$last` | Último valor del grupo | `{ $last: "$cliente" }` | `{"$last": "$cliente"}` |
| `$project` | Transformar o seleccionar campos | `{ $project: { cliente: 1, total: 1 } }` | `{"$project": {"cliente":1, "total":1}}` |
| `$sort` | Ordenar resultados | `{ $sort: { total: -1 } }` | `{"$sort": {"total": -1}}` |
| `$limit` | Limitar resultados | `{ $limit: 5 }` | `{"$limit": 5}` |
| `$unwind` | Separar arrays en documentos individuales | `{ $unwind: "$productos" }` | `{"$unwind": "$productos"}` |

---

### 3. Pipeline de agregación básico

Supongamos que tenemos documentos de cotizaciones con este formato:

```json
{
  "id_rablo": "120383",
  "cliente": "fblanco",
  "estatus": "Pendiente",
  "fecha_creacion": ISODate("2025-10-20"),
  "productos": [
    {"nombre": "Arduino Uno", "cantidad": 2, "precio": 150},
    {"nombre": "Sensor DHT11", "cantidad": 5, "precio": 30}
  ]
}
```

```python
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["miBaseDeDatos"]
collection = db["cotizaciones"]

pipeline = [
    # Separar los productos de cada cotización
    {"$unwind": "$productos"},

    # Agrupar por estatus
    {"$group": {
        "_id": "$estatus",
        "total_cantidad": {"$sum": "$productos.cantidad"},
        "total_precio": {"$sum": {"$multiply": ["$productos.cantidad", "$productos.precio"]}}
    }},

    # Ordenar de mayor a menor cantidad
    {"$sort": {"total_cantidad": -1}}
]

for result in collection.aggregate(pipeline):
    print(result)